In [1]:
import json
from pprint import pprint
import boto3
import pathlib


In [3]:
image_folder = pathlib.Path('Images')
output_folder = pathlib.Path('OCR')
output_folder.mkdir(exist_ok=True)

In [14]:
#(DO NOT MODIFY)

def extract_text_from_image(filename: pathlib.Path):
    textract = boto3.client('textract', region_name='us-west-2')
    response = textract.analyze_document(
        Document={
            'Bytes': filename.read_bytes()
        },
        FeatureTypes=['TABLES', 'FORMS', ]
    )
    
    result = {
        "Pages": len(response['Blocks']),
        "Blocks": response['Blocks']
    }
    #return json dumps
   
    
    return json.dumps(result, indent=4, sort_keys=True, default=str)

In [15]:
# (DO NOT MODIFY)
# making a loop to go through all the images in the folder
for filename in image_folder.iterdir():
    # Extract the text from the image
    text = extract_text_from_image(filename)
    # Save the text to a json file
    output_file = output_folder / (filename.stem + '.json')
    output_file.write_text(text)
    

In [24]:
# this one works, but I want to make it work for all the json files in the folder
#DO NOT MODIFY

testimg1 = pathlib.Path('OCR/testimage1.json')
testlist1 = []
with open(testimg1) as f:
    data = json.load(f)
    for i in data['Blocks']:
        if i['BlockType'] == 'LINE' or i['BlockType'] == 'WORD':
            testlist1.append(i["Text"])

pprint(testlist1)


['11/10/20',
 '11 10 20',
 '21',
 '95940',
 '5.9',
 'KL',
 '1354/00 2',
 '2',
 'I',
 'I',
 '11 10 20',
 '11',
 '10/20',
 '21',
 '95861',
 'TC',
 'A',
 '3460 no',
 '1',
 '3',
 '11 10.20',
 '10/20',
 '21',
 '95927',
 'T.C.',
 'A',
 '2664 0.0',
 '1',
 '4',
 '11 10 20',
 '11/10/20 21 95938',
 'TC',
 '7013 0.0',
 '5',
 '11',
 '10 20',
 '11',
 '10 20',
 '21',
 '95937',
 'C',
 'A',
 '1890',
 '00',
 '1',
 '6',
 '95955',
 'TO',
 'A',
 '4216',
 '1',
 '11/10/20',
 '11 10 20',
 '21',
 '95940',
 '5.9',
 'KL',
 '1354/00',
 '2',
 '2',
 'I',
 'I',
 '11 10',
 '20',
 '11',
 '10/20',
 '21',
 '95861',
 'TC',
 'A',
 '3460',
 'no',
 '1',
 '3',
 '11',
 '10.20',
 '10/20',
 '21',
 '95927',
 'T.C.',
 'A',
 '2664',
 '0.0',
 '1',
 '4',
 '11',
 '10 20',
 '11/10/20',
 '21',
 '95938',
 'TC',
 '7013',
 '0.0',
 '5',
 '11',
 '10 20',
 '11',
 '10 20',
 '21',
 '95937',
 'C',
 'A',
 '1890',
 '00',
 '1',
 '6',
 '95955',
 'TO',
 'A',
 '4216',
 '1']


In [ ]:
#loop through all the json files in the folder and extract the text from the blocks into 5 lists
testlist1 = []
testlist2 = []
testlist3 = []
testlist4 = []
testlist5 = []

#make a path to the OCR folder
OCR = pathlib.Path('OCR')

for filename in (OCR).iterdir():
    with open(filename) as f:
        data = json.load(f)
        for i in data['Blocks']:
            if i['BlockType'] == 'LINE' or i['BlockType'] == 'WORD':
                if filename == 'testlist1.json':
                    testlist1.append(i["Text"])
                elif filename == 'testlist2.json':
                    testlist2.append(i['Text'])
                elif filename == 'testlist3.json':
                    testlist3.append(i['Text'])
                elif filename == 'testlist4.json':
                    testlist4.append(i['Text'])
                elif filename == 'testlist5.json':
                    testlist5.append(i['Text'])
                
#why are my lists empty?
pprint(testlist1)



In [27]:
#loop through all the json files in the folder and extract the text from the blocks into 5 lists
testlist1 = []
testlist2 = []
testlist3 = []
testlist4 = []
testlist5 = []

#make a path to the OCR folder
OCR = pathlib.Path('OCR')

for filename in (OCR).iterdir():
    with open(filename) as f:
        data = json.load(f)
        for i in data['Blocks']:
            if i['BlockType'] == 'LINE' or i['BlockType'] == 'WORD':
                if filename == 'testimage1.json':
                    testlist1.append(i["Text"])
                elif filename == 'testimage2.json':
                    testlist2.append(i['Text'])
                elif filename == 'testimage3.json':
                    testlist3.append(i['Text'])
                elif filename == 'testimage4.json':
                    testlist4.append(i['Text'])
                elif filename == 'testimage5.json':
                    testlist5.append(i['Text'])
                
#why are my lists empty?
pprint(testlist1)



[]


In [25]:

#make a loop to go through all the json files in the folder and extract the text from the blocks into 5 lists
testlist1 = []
testlist2 = []
testlist3 = []
testlist4 = []
testlist5 = []

#make a path to the OCR folder
OCR = pathlib.Path('OCR')

for filename in (OCR).iterdir():
    with open(filename) as f:
        data = json.load(f)
        for i in data['Blocks']:
            if i['BlockType'] == 'LINE' or i['BlockType'] == 'WORD':
                if filename == 'testlist1.json':
                    testlist1.append(i["Text"])
                elif filename == 'testlist2.json':
                    testlist2.append(i['Text'])
                elif filename == 'testlist3.json':
                    testlist3.append(i['Text'])
                elif filename == 'testlist4.json':
                    testlist4.append(i['Text'])
                elif filename == 'testlist5.json':
                    testlist5.append(i['Text'])


print(testlist1)

[]
